# Logistic Regression in TensorFlow [20 points]

## Building the Computational Graph [5 pts]

In [1]:
import matplotlib as plt
import numpy as np

In [2]:
def sigmoid(s):
    return np.exp(s) / (1+ np.exp(s))

In [3]:
def evaluate_prediction (w, b, x, label): # evaluate accuracy rate
    prob = sigmoid(np.dot(x, w) + b)
    predict = prob > 0.5
    correct = np.where(predict == label)[0].size # how many correct
    return correct / np.shape(predict)[0] # correct estimation out of all sample

In [4]:
#!pip install tensorflow==1.14.0
import tensorflow as tf
print(tf.__version__)
#tf.reset_default_graph()

In [1]:
def buildGraph(beta1, beta2, epsilon, alpha):
    w = tf.Variable(tf.random.truncated_normal([28 * 28, 1], mean=0, stddev=0.5, dtype=tf.float32, trainable=True))
    b = tf.Variable(tf.zeros(1), name='b', dtype=tf.float32, trainable=True)

    x = tf.placeholder(tf.float32, [None, 784])
    y = tf.placeholder(tf.float32, [None, 1])
    reg = tf.placeholder(tf.float32)

    logits = tf.matmul(x, w) + b

    loss = tf.losses.sigmoid_cross_entropy(multi_class_labels=y, logits=logits) + reg * tf.nn.l2_loss(w) #latter term returns half the l2 norm of w

    optimizer = tf.compat.v1.train.AdamOptimizer(alpha, beta1, beta2, epsilon).minimize(loss)

    return w, b, x, y, reg, loss, optimizer


In [ ]:
def plot_curves2(reg, alpha, batchsize, b1, b2, ep, train_acc, train_loss, val_acc, val_loss, test_acc, test_loss, plot_test):
    n = len(train_acc)
    plt.figure(figsize=(20,10))
    plt.title('TF Model Loss for alpha={}, reg={}, batch={}, b1={}, b2={}, eps={}'.format(alpha,reg,batchsize,b1,b2,ep))
    plt.plot(range(1,1+n), train_loss, label="Train")
    plt.plot(range(1,1+n), val_loss, label="Validation")
    if plot_test:
        plt.plot(range(1,1+n), test_loss, label="Test")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.legend(loc='best')
    plt.show()
    print("Final Training Loss: {}".format(train_loss[-1]))
    print("Final Validation Loss: {}".format(val_loss[-1]))
    if plot_test:
        print("Final Testing Loss: {}".format(test_loss[-1]))

    plt.figure(figsize=(20,10))
    plt.title('TF Model Accuracy for alpha={}, reg={}, batch={}, b1={}, b2={}, eps={}'.format(alpha,reg,batchsize,b1,b2,ep))
    plt.plot(range(1,1+n), train_acc, label="Train")
    plt.plot(range(1,1+n), val_acc, label="Validation")
    if plot_test:
        plt.plot(range(1,1+n), test_acc, label="Test")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.legend(loc='best')
    plt.show()
    print("Final Training Accuracy: {}".format(train_acc[-1]))
    print("Final Validation Accuracy: {}".format(val_acc[-1]))
    if plot_test:
        print("Final Testing Loss: {}".format(test_acc[-1]))

In [ ]:
def tf_sgd(train_data, valid_data, test_data, trainTarget,validTarget, testTarget, epochs, bs, reg, lr, b1, b2, ep, plot_test):
    tf.compat.v1.set_random_seed(421)
    W, b, pred, y, loss, optim, d, r = buildGraph(lr,b1,b2,ep)
    init_op = tf.compat.v1.global_variables_initializer()

    N = train_data.shape[0]
    batches = int(N/bs)
    train_accs, val_accs, train_losses, val_losses, test_accs, test_losses = [], [], [], [], [], []

    train_data = np.reshape(train_data, (3500, -1))
    valid_data = np.reshape(valid_data, (100, -1))
    test_data = np.reshape(test_data, (145, -1))

    with tf.compat.v1.Session() as sess:
        sess.run(init_op)

        for e in range(0, epochs, 1):
            shuffled = np.random.permutation(N)
            train_data, trainTarget = train_data[shuffled], trainTarget[shuffled]

            train_loss = 0.0
            train_acc = 0.0

            for iter in range(0, batches, 1):
                iter *= bs
                _, train_predicts, batch_loss = sess.run([optim, pred, loss], feed_dict={r:reg, d:train_data[iter:iter+bs], y:trainTarget[iter:iter+bs]})
                batch_acc = evaluate_prediction(train_predicts, trainTarget[iter:iter+bs])
                train_loss += batch_loss
                train_acc += batch_acc

            train_acc /= batches
            train_loss /= batches

            valid_predicts,valid_loss = sess.run([pred, loss], feed_dict={r:reg, d:valid_data, y:validTarget})
            valid_acc = evaluate_prediction(valid_predicts, validTarget)

            test_predicts,test_loss = sess.run([pred, loss], feed_dict={r:reg, d:test_data, y:testTarget})
            test_acc = evaluate_prediction(test_predicts, testTarget)
                
            train_accs.append(train_acc)
            train_losses.append(train_loss)
            val_accs.append(valid_acc)
            val_losses.append(valid_loss)
            test_accs.append(test_acc)
            test_losses.append(test_loss)

        sess.close()

    plot_curves2(reg, lr, bs, b1, b2, ep, train_accs, train_losses, val_accs, val_losses, test_accs, test_losses, plot_test)   

In [ ]:
train_data, valid_data, test_data, trainTarget, validTarget, testTarget = loadData()

epochs = 700
batchsize = 500
reg = 0
lr = 0.001
b1 = 0.9
b2 = 0.999
ep = 1e-08

tf_sgd(train_data, valid_data, test_data, trainTarget,validTarget, testTarget, epochs, batchsize, reg, lr, b1, b2, ep, 0)

NameError: name 'loadData' is not defined